In [60]:
# https://stackoverflow.com/questions/34322471/sqlalchemy-engine-connection-and-session-difference

In [61]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options


from bs4 import BeautifulSoup
import time
import pandas as pd
from pandas import DataFrame as DF
from IPython.display import display

from sqlalchemy import *
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker,relationship,backref


import smtplib
from email.mime.text import MIMEText


from urllib.request import urlopen
import json 

In [62]:
def get_browser():
    options = Options()
    options.headless = True
    _browser_profile = webdriver.FirefoxProfile()
    _browser_profile.set_preference("dom.webnotifications.enabled", False)
    executable_path='/media/disk3/feynman52/dummy/SQLAlchemy/591/geckodriver'
    browser = webdriver.Firefox(executable_path=executable_path,
                                options=options,
                                firefox_profile=_browser_profile)
    return browser

def get_page(browser,page=''):
    browser.get(page)
    time.sleep(1)
    
def click_button(browser,xpath=''):
    dummy=browser.find_element_by_xpath(xpath)
    dummy.click()
    time.sleep(1)
    
def send_keys(browser,xpath='',value=''):
    dummy=browser.find_element_by_xpath(xpath)
    dummy.send_keys(value)
    time.sleep(1)
    
def get_session(engine):
    Session = sessionmaker(bind = engine)
    session = Session()
    return session
    
def add_to_table(session,item):
    session.add(item)
    session.commit()
    
def delete_from_table(session,item):
    session.delete(item)
    session.commit()
    
def sql_to_df(engine,query=''):
    df = pd.read_sql_query(query, engine)
    return df

def get_engine(db_name=''):
    db_dir='/media/disk3/feynman52/See26/crawler-ryh'
    db_url = 'sqlite:///'+db_dir+'/'+db_name
    engine = create_engine(db_url, echo = False)
    return engine
    

    

# get articles

In [73]:
engine_article = get_engine(db_name='articles.db')

Base = declarative_base()
class Article(Base):
    __tablename__ = 'article'
    article_id = Column(Integer, primary_key=True)
    article_user = Column(String)
    article_group = Column(String)
    article_title = Column(String)
    article_time = Column(String)
    article_weekday = Column(Integer)
    article_ip = Column(String)   
    article_url = Column(String)    
    
    

class Push(Base):
    __tablename__ = 'push'
    push_id = Column(Integer, primary_key=True)
    push_user = Column(String)
    push_tag = Column(String)
    push_time = Column(String)
    push_content = Column(String)
    push_weekday = Column(Integer)
    push_ip = Column(String)
    
    article_id = Column(Integer, ForeignKey('article.article_id'))
    article = relationship("Article", back_populates = "push")
    
Article.push=relationship("Push", order_by = Push.push_id, back_populates = "article")
Base.metadata.create_all(engine_article)

# Push.__table__.drop(engine_article)
# Article.__table__.drop(engine_article)


In [102]:
def append_url_to_file(url=''):
    with open("error.txt", "a") as myfile:
        myfile.write("%s\n"%(url))
        
# append_url_to_file(url='www.com')

In [64]:
def get_current_page(browser,url):
    get_page(browser,page=url)
    pageSource = browser.page_source  
    current_page = BeautifulSoup(pageSource)  
    return current_page

In [94]:
def get_article(current_page,article_url):
    article_meta_value=current_page.find_all('span',{'class':'article-meta-value'})
    article_meta_value=[x.string for x in article_meta_value]
    article_user=article_meta_value[0].split(' ')[0]
    article_group=article_meta_value[1]
    article_title=article_meta_value[2]
    article_time=str(pd.to_datetime(article_meta_value[3]))
    article_weekday=pd.to_datetime(article_time).weekday()
    article_ip=current_page.find_all('span',{'class':'f2'})
    article_ip=[x.string for x in article_ip if x.string!=None and '來自' in x.string]
    
    try: 
        article_ip=article_ip[0]
        article_ip=article_ip[article_ip.find('來自: ')+len('來自: '):]
        article_ip=article_ip.split(' ')[0].replace('\n','')
    except:
        article_ip='no_article_ip'
    

    article=Article(
        article_user = article_user,
        article_group = article_group,
        article_title = article_title,
        article_time = article_time,
        article_weekday = article_weekday,
        article_ip = article_ip,
        article_url = article_url
    )
    
    return article

article_url='https://www.ptt.cc/bbs/Gossiping/M.1411274605.A.27B.html'
get_article(get_current_page(browser,article_url),article_url)

['http://www.ptt.cc/bbs/Gossiping/M.1411274605.A.27B.html', '                                                      09/21 12:43', '藍巴腦蛆不是最愛說對手支持者都「一高二低」？            09/21 12:44', 'XDDD                                                 09/21 12:44']


ValueError: ('Unknown string format:', 'XDDD                                                 09/21 12:44')

# get pushs

In [85]:
def get_pushs(current_page,article_time):
    pushs=current_page.find_all('div',{'class':'push'})
    push_objects=[]
    for push in pushs:
        push=push.find_all('span')
        
        if len(push)==0: continue
        
        push=[x.text.replace('\n','') for x in push]
        push_tag=push[0].replace(' ','')

        push_user=push[1]

        push_content=push[2].replace(': ','')

        push_ipdt=push[3]

        push_ip=push_ipdt[:-len('07/23 15:53')].replace(' ','')
        push_ip=push_ip if len(push_ip)>0 else 'no_push_ip'

        push_time=' '.join([article_time[:4],push_ipdt[-len('07/23 15:53'):]])
        push_time=str(pd.to_datetime(push_time))

        push_weekday=pd.to_datetime(push_time).weekday()

        push=Push(
            push_user = push_user,
            push_tag = push_tag,
            push_time = push_time,
            push_content = push_content,
            push_weekday = push_weekday,
            push_ip = push_ip
        )
        push_objects.append(push)
        
    return push_objects
    

In [67]:
browser=get_browser()
get_page(browser,page='https://www.ptt.cc/bbs/Gossiping/index.html')
click_button(browser,xpath='/html/body/div[2]/form/div[1]/button')
session=get_session(engine_article)

In [68]:

# xs = session.query(Push).filter(Push.article_id==None)
# for x in xs:
#     delete_from_table(session,x)

In [87]:
def get_urls(engine_url,engine_article):
    new_urls=sql_to_df(engine_url,query='select * from urls').url.tolist()
    old_urls=sql_to_df(engine_article,query='select * from article').article_url.tolist()
    new_urls_after_remove=list(set(new_urls)-set(old_urls))
    new_urls_after_remove.sort()
    return new_urls_after_remove

engine_url=get_engine(db_name='urls.db')
urls=get_urls(engine_url,engine_article)
len(urls)

720732

In [86]:
old_urls=sql_to_df(engine_article,query='select * from article').article_url.tolist()
current=6000+len(old_urls)
total=len(urls)
for url in ['https://www.ptt.cc//bbs/Gossiping/M.1407342567.A.EF2.html']: #urls[current:]:
    
    print('---------------')
    print(url)
    print('%s/%s'%(current,total))
    current+=1
    
    current_page=get_current_page(browser,url=url)
    
    article=get_article(current_page,url)
    pushs=get_pushs(current_page,article_time=article.article_time)
    article.push = pushs
    
#     add_to_table(session,article)
    

---------------
https://www.ptt.cc//bbs/Gossiping/M.1407342567.A.EF2.html
6000/720852


In [106]:

query='''
select * from article
'''

sql_to_df(engine_article,query=query)

,article_id,article_user,article_group,article_title,article_time,article_weekday,article_ip,article_url
0,1,Bignana,Gossiping,[公告] 水桶名單,2014-06-28 21:09:57,5,114.27.16.120,https://www.ptt.cc//bbs/Gossiping/M.1403961000...
1,2,Bignana,Gossiping,[公告] 水桶名單,2014-06-28 21:56:38,5,220.137.3.66,https://www.ptt.cc//bbs/Gossiping/M.1403963801...
2,3,Lavchi,Gossiping,[公告] 今日水桶 ^.<,2014-06-28 22:05:28,5,114.26.64.158,https://www.ptt.cc//bbs/Gossiping/M.1403964331...
3,4,Lavchi,Gossiping,[公告] 水桶公告 part2,2014-06-28 23:31:27,5,114.26.64.158,https://www.ptt.cc//bbs/Gossiping/M.1403969491...
4,5,seabox,Gossiping,[公告] 水桶,2014-06-28 23:52:56,5,60.244.184.106,https://www.ptt.cc//bbs/Gossiping/M.1403970780...
5,6,talk520,Gossiping,[公告] 水桶,2014-06-29 14:53:32,6,114.36.216.133,https://www.ptt.cc//bbs/Gossiping/M.1404024815...
6,7,Lavchi,Gossiping,[公告] 今日水桶,2014-06-29 20:27:24,6,218.161.52.214,https://www.ptt.cc//bbs/Gossiping/M.1404044856...
7,8,seabox,Gossiping,[公告] 水桶,2014-06-29 23:47:57,6,60.244.184.106,https://www.ptt.cc//bbs/Gossiping/M.1404056880...
8,9,Bignana,Gossiping,[公告] 水桶,2014-06-30 06:20:21,0,118.171.65.123,https://www.ptt.cc//bbs/Gossiping/M.1404080424...
9,10,Lavchi,Gossiping,[公告] 退伍前的水桶公告來囉!,2014-06-30 22:45:38,0,218.161.52.214,https://www.ptt.cc//bbs/Gossiping/M.1404139541...


In [78]:

query='''
select * from push
'''
sql_to_df(engine_article,query=query)


,push_id,push_user,push_tag,push_time,push_content,push_weekday,push_ip,article_id
0,1,arlen26,噓,2014-06-26 01:35:00,:在大陸什麼聲音也都有，只是沒激烈到這程度,3,no_push_ip,1
1,2,arlen26,→,2014-06-26 01:35:00,:而且現實中民眾關心政治的并不多,3,no_push_ip,1
2,3,arlen26,→,2014-06-26 01:36:00,:反而是自稱自由民主的某地容不下別人的聲音,3,no_push_ip,1
3,4,meredith001,推,2014-06-28 21:10:00,:桶的好 五毛滾出PTT,5,no_push_ip,1
4,5,gh34163,推,2014-06-28 21:10:00,:ㄏㄏ,5,no_push_ip,1
5,6,Ives20130,推,2014-06-28 21:10:00,:讚,5,no_push_ip,1
6,7,rabbina,→,2014-06-28 21:10:00,:本國人推,5,no_push_ip,1
7,8,a12678922,推,2014-06-28 21:10:00,:你有吻過5樓嗎,5,no_push_ip,1
8,9,kairi5217,推,2014-06-28 21:11:00,:26真逗 什麼聲音都有 是因為沒關廁所門嗎?,5,no_push_ip,1
9,10,gx9900824,推,2014-06-28 21:11:00,:共匪滾回去!!!!,5,no_push_ip,1


# ip to country

In [41]:
# def ip2country(ip=''):
#     url = 'https://ipinfo.io/' + ip + '/json'
#     res = urlopen(url)
#     data = json.load(res)
#     country=data['country']
#     return country
        
# ip='180.204.85.233'
# ip2country(ip=ip)